
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import numpy as np
import pandas as pd
file_path = "data/Corona_NLP_train.csv"
data = pd.read_csv(file_path, encoding='latin1')

data.head(10)

# filtered_data

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [9]:
filtered_data = data[data['Sentiment'] != 'Neutral']
filtered_data

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
...,...,...,...,...,...,...
41150,44949,89901,OHIO,14-04-2020,I never that weÂd be in a situation &amp; wor...,Positive
41151,44950,89902,NaN,14-04-2020,@MrSilverScott you are definitely my man. I fe...,Extremely Positive
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive


In [10]:
# Step 3: Convert `Sentiment` to binary (1 for positive, 0 for negative)
# this is creating y 
filtered_data['Sentiment'] = filtered_data['Sentiment'].map(
    lambda sentiment: 1 if sentiment in ['Positive', 'Extremely Positive'] else 0
)
filtered_data

C:\Users\saber\AppData\Local\Temp\ipykernel_16684\673604603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Sentiment'] = filtered_data['Sentiment'].map(


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,1
...,...,...,...,...,...,...
41150,44949,89901,OHIO,14-04-2020,I never that weÂd be in a situation &amp; wor...,1
41151,44950,89902,NaN,14-04-2020,@MrSilverScott you are definitely my man. I fe...,1
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,0
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,1


In [2]:
25/40

0.625

In [11]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.
import numpy as np
import pandas as pd

file_path = "data/Corona_NLP_train.csv"
data = pd.read_csv(file_path, encoding='latin1')

data.head(10)
# Filter out rows where `Sentiment` is Neutral
filtered_data = data[data['Sentiment'] != 'Neutral']

# Step 3: Convert `Sentiment` to binary (1 for positive, 0 for negative)
filtered_data['Sentiment'] = filtered_data['Sentiment'].map(
    lambda sentiment: 1 if sentiment in ['Positive', 'Extremely Positive'] else 0
)

# Step 4: Define X (OriginalTweet) and Y (Sentiment)
# Make sure that all your data is numpy arrays and not pandas dataframes or series.
# X = XXX
# Y = YYY
X = filtered_data['OriginalTweet'].values
Y = filtered_data['Sentiment'].values

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.
# X_train = XXX
# Y_train = YYY
# X_test = XXX
# Y_test = YYY
# X_valid = XXX
# Y_valid = YYY
# n = len(X)
# train_end = int(0.6 * n)
# test_end = int(0.75 * n)

# X_train, Y_train = X[:train_end], Y[:train_end]
# X_test, Y_test = X[train_end:test_end], Y[train_end:test_end]
# X_valid, Y_valid = X[test_end:], Y[test_end:]
from sklearn.model_selection import train_test_split

# First split: Training set (60%) and remaining set (40%)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)

# Second split: Test set (15%) and validation set (25%)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_temp, Y_temp, test_size=25/40, random_state=42)
# Output the shapes of the splits for verification
print("Train set:", X_train.shape, Y_train.shape)
print("Test set:", X_test.shape, Y_test.shape)
print("Validation set:", X_valid.shape, Y_valid.shape)

Train set: (20066,) (20066,)
Test set: (5016,) (5016,)
Validation set: (8362,) (8362,)


C:\Users\saber\AppData\Local\Temp\ipykernel_16684\2072013047.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Sentiment'] = filtered_data['Sentiment'].map(


In [19]:
model

Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_features=5000, stop_words='english')),
                ('classifier',
                 LogisticRegression(max_iter=500, random_state=42))])

In [13]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


# Define the pipeline
model = Pipeline([
    #  Converts raw text into a numerical representation (bag-of-words model).
    ('vectorizer', CountVectorizer(max_features=5000, stop_words='english')),
    # A linear classifier that predicts Y=0 (negative sentiment) or Y=1 (positive sentiment).
    ('classifier', LogisticRegression(max_iter=500, solver='lbfgs', random_state=42))
])

# Train the model on the training data (trains the entire pipeline)
model.fit(X_train, Y_train)

# Output to confirm the model is trained
print("Model training complete.")
# model = XXX

Model training complete.


This code above uses a pipeline to streamline the process of preprocessing text data and training a machine learning model. A pipeline allows you to define a sequence of steps (e.g., preprocessing, vectorization, and classification) and treat them as a single model for training and prediction.



In [14]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

# precision_0 = XXX
# precision_1 = XXX
# recall_0 = XXX
# recall_1 = XXX
from sklearn.metrics import precision_score, recall_score

# Predict on the test set
Y_test_pred = model.predict(X_test)

# Calculate precision and recall for both classes
precision_0 = precision_score(Y_test, Y_test_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_test_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_test_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_test_pred, pos_label=1)

# Output results
print(f"Precision (class 0): {precision_0}")
print(f"Precision (class 1): {precision_1}")
print(f"Recall (class 0): {recall_0}")
print(f"Recall (class 1): {recall_1}")

Precision (class 0): 0.8390201224846894
Precision (class 1): 0.8673992673992674
Recall (class 0): 0.8412280701754385
Recall (class 1): 0.8654970760233918


In [15]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    
    # return XXX
    
    # Step 1: Get probabilities for class 1
    probabilities = model.predict_proba(X)[:, 1]  # P(Y=1)

    # Step 2: Apply the threshold to make predictions
    predictions = (probabilities >= threshold).astype(int)

    # Step 3: Assign costs based on predictions and true labels
    costs = np.where(
        (Y == 1) & (predictions == 0),  # False negative
        1,  # Cost = 1
        np.where(
            (Y == 0) & (predictions == 1),  # False positive
            5,  # Cost = 5
            0  # Correct classification if the prediction mateches the label
        )
    )

    # Step 4: Return the average cost
    return costs.mean()

In [16]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
# optimal_threshold = XXX
# cost_at_optimal_threshold = XXX

# Step 1: Define a range of thresholds
thresholds = np.arange(0.0, 1.01, 0.01)  # Thresholds from 0.0 to 1.0 in steps of 0.01

# Step 2: Initialize variables to track the optimal threshold and its cost
optimal_threshold = None
min_cost = float('inf')

# Step 3: Loop through thresholds and compute cost
for threshold in thresholds:
    current_cost = cost(model, threshold, X_test, Y_test)
    if current_cost < min_cost:
        min_cost = current_cost
        optimal_threshold = threshold

# Step 4: Store results
cost_at_optimal_threshold = min_cost

# Output the results
print(f"Optimal threshold: {optimal_threshold}")
print(f"Cost at optimal threshold: {cost_at_optimal_threshold}")


Optimal threshold: 0.79
Cost at optimal threshold: 0.29366028708133973


Hoeffding's epsilon and confidence interval


--> assumes a fixed range for the cost which can be oldfashion 


In [17]:

# # Part 6

# cost_at_optimal_threshold_valid = XXX
# cost_interval_valid = XXX



import numpy as np

# Step 1: Evaluate the cost on the validation set
cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

# Step 2: Define parameters for Hoeffding's inequality
n_valid = len(Y_valid)  # Number of validation samples
delta = 0.01  # 99% confidence level
epsilon = np.sqrt(np.log(2 / delta) / (2 * n_valid))  # Hoeffding's epsilon

# Step 3: Calculate the confidence interval
cost_interval_valid = (
    max(0, cost_at_optimal_threshold_valid - epsilon),  # Lower bound (ensure non-negative)
    min(1, cost_at_optimal_threshold_valid + epsilon)   # Upper bound (ensure <= 1)
)

# Output results
print(f"Cost on validation set at optimal threshold: {cost_at_optimal_threshold_valid}")
print(f"Confidence interval (99%): {cost_interval_valid}")

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

Cost on validation set at optimal threshold: 0.2958622339153313
Confidence interval (99%): (np.float64(0.2780630985913333), np.float64(0.31366136923932925))


Bennett's inequality and empirical variance of C


---> is more accuret because it takes in to account the variance of the cost 

In [18]:

# # Part 7

# variance_of_C = XXX
# interval_of_C = XXX
import numpy as np

# Step 1: Calculate C for each validation sample
probabilities_valid = model.predict_proba(X_valid)[:, 1]  # P(Y=1)
predictions_valid = (probabilities_valid >= optimal_threshold).astype(int)

C = (1 - predictions_valid) * Y_valid + 5 * (1 - Y_valid) * predictions_valid

# Step 2: Calculate empirical variance of C
mean_C = C.mean()
variance_of_C = ((C - mean_C) ** 2).mean()

# Step 3: Apply Bennett's inequality
n_valid = len(Y_valid)  # Number of validation samples
delta = 0.01  # 99% confidence level
b = 5  # Maximum value of C
x = np.log(2 / delta) / (n_valid * variance_of_C)

# Calculate epsilon using Bennett's inequality
epsilon = b ** 2 * (x - np.log(1 + x))

# Step 4: Confidence interval
interval_of_C = (
    max(0, mean_C - epsilon),  # Lower bound
    min(5, mean_C + epsilon)   # Upper bound
)

# Output results
print(f"Variance of C: {variance_of_C}")
print(f"Confidence interval (99%) using Bennett's inequality: {interval_of_C}")


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Variance of C: 0.7177752730559043
Confidence interval (99%) using Bennett's inequality: (np.float64(0.29585249897825755), np.float64(0.295871968852405))
